# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [122]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium
import json

# Import API key
from api_keys import geoapify_key

In [123]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,292.78,43,75,2.57,US,1686371015
1,1,axim,4.8699,-2.2405,298.49,85,85,3.68,GH,1686371015
2,2,ribeira grande,38.5167,-28.7000,291.36,78,14,4.02,PT,1686371015
3,3,sirte,31.2089,16.5887,295.07,62,77,4.47,LY,1686371015
4,4,tamanrasset,22.7850,5.5228,300.10,12,0,2.06,DZ,1686371016


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [124]:
# Create a map centered at a specific location
map = folium.Map(location=[0, 0], zoom_start=2)

# Iterate over the rows of the city_data_df DataFrame
for index, row in city_data_df.iterrows():
    # Get the latitude, longitude, and humidity values for each city
    lat = row['Lat']
    lon = row['Lng']
    humidity = row['Humidity']
    
    # Create a CircleMarker for each city, with the size based on humidity
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  # Adjust the scale as per your preference
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map)

# Display the map
map

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [125]:
# Define your ideal weather conditions
ideal_conditions = {
    'Max Temp': 345,  # Maximum temperature in Fahrenheit
    'Humidity': 50,  # Humidity percentage
    'Cloudiness': 0,  # Cloudiness percentage
    'Wind Speed': 4.5  # Wind speed in mph
}

# Filter the DataFrame based on the ideal conditions
filtered_df = city_data_df[
    (city_data_df['Max Temp'] <= ideal_conditions['Max Temp']) &
    (city_data_df['Humidity'] <= ideal_conditions['Humidity']) &
    (city_data_df['Cloudiness'] <= ideal_conditions['Cloudiness']) &
    (city_data_df['Wind Speed'] <= ideal_conditions['Wind Speed'])
]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
ideal_weather_data = filtered_df.sample(20)
ideal_weather_data


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
125,125,thana bhawan,29.5864,77.4167,311.59,23,0,2.93,IN,1686371038
345,345,songyang,34.4553,113.0281,305.84,18,0,3.38,CN,1686371088
434,434,maun,-19.9833,23.4167,289.13,29,0,4.12,BW,1686371106
12,12,alice springs,-23.7000,133.8833,294.90,16,0,4.12,AU,1686371017
428,428,pokhara,28.2333,83.9833,305.63,29,0,2.92,NP,1686371105
93,93,ghat,24.9647,10.1728,302.70,24,0,3.51,LY,1686371032
271,271,broken hill,-31.9500,141.4333,288.59,42,0,1.84,AU,1686371062
368,368,geraldton,-28.7667,114.6000,294.38,49,0,1.54,AU,1686370980
206,206,manzini,-26.2500,31.5000,285.34,40,0,2.56,SZ,1686371058
521,521,yuma,32.7253,-114.6244,302.03,18,0,3.09,US,1686371125


### Step 3: Create a new DataFrame called `hotel_df`.

In [126]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a new DataFrame with required columns
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display the data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,tamanrasset,DZ,22.7850,5.5228,12,
12,alice springs,AU,-23.7000,133.8833,16,
93,ghat,LY,24.9647,10.1728,24,
125,thana bhawan,IN,29.5864,77.4167,23,
129,port macquarie,AU,-31.4333,152.9167,50,
180,panjab,AF,34.3880,67.0233,29,
191,hami,CN,42.8000,93.4500,10,
206,manzini,SZ,-26.2500,31.5000,40,
221,parun,AF,35.4207,70.9221,35,
243,al jahra',KW,29.3375,47.6581,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [127]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "type": "hotel",
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {radius}@{lon},{lat}"
    params["bias"] = f"proximity: {lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
    #name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        city_data_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tamanrasset - nearest hotel: 


alice springs - nearest hotel: 
ghat - nearest hotel: 
thana bhawan - nearest hotel: 
port macquarie - nearest hotel: 
panjab - nearest hotel: 
hami - nearest hotel: 
manzini - nearest hotel: 
parun - nearest hotel: 
al jahra' - nearest hotel: 
medina - nearest hotel: 
broken hill - nearest hotel: 
badger - nearest hotel: 
mount isa - nearest hotel: 
nelson bay - nearest hotel: 
rakops - nearest hotel: 
songyang - nearest hotel: 
geraldton - nearest hotel: 
kashgar - nearest hotel: 
pokhara - nearest hotel: 
maun - nearest hotel: 
al kharijah - nearest hotel: 
yasnyy - nearest hotel: 
yuma - nearest hotel: 
singrauli - nearest hotel: 
ulan bator - nearest hotel: 
abha - nearest hotel: 


,City,Country,Lat,Lng,Humidity,Hotel Name
4,tamanrasset,DZ,22.7850,5.5228,12,
12,alice springs,AU,-23.7000,133.8833,16,
93,ghat,LY,24.9647,10.1728,24,
125,thana bhawan,IN,29.5864,77.4167,23,
129,port macquarie,AU,-31.4333,152.9167,50,
180,panjab,AF,34.3880,67.0233,29,
191,hami,CN,42.8000,93.4500,10,
206,manzini,SZ,-26.2500,31.5000,40,
221,parun,AF,35.4207,70.9221,35,
243,al jahra',KW,29.3375,47.6581,13,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [128]:
%%capture --no-display

# Configure the map plot
# # Create a map centered at a specific location
map2 = folium.Map(location=[0, 0], zoom_start=2)

# Iterate over the rows of the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude, longitude, and humidity values for each city
    lat = row['Lat']
    lon = row['Lng']
    humidity = row['Humidity']
    hotel_name = row['Hotel Name']
    country = row['Country']
    
    # Create a CircleMarker for each city, with the size based on humidity
    folium.CircleMarker(
        location=[lat, lon],
        radius=humidity/10,  # Adjust the scale as per your preference
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        tooltip=f"City: {row['City']}, Country: {country}<br>Hotel: {hotel_name}"
    ).add_to(map2)

# Display the map
map2